<a href="https://colab.research.google.com/github/kirill-kozlitin/VTB/blob/main/%D0%9A%D0%BE%D0%B2%D0%BE%D1%80%D0%BA%D0%B8%D0%BD%D0%B3_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%BA%D0%BE%D0%BD%D0%BA%D1%83%D1%80%D0%B5%D0%BD%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Установка библиотек на колаб

In [23]:
!apt-get install -y cmake libtool
!git clone https://github.com/uber/h3.git
%cd h3
!mkdir build
%cd build
!cmake ..
!make
!make install
!rm -rf h3

^C
Cloning into 'h3'...
remote: Enumerating objects: 22785, done.
remote: Counting objects: 100% (22785/22785), done.
remote: Compressing objects: 100% (4850/4850), done.
^C
[Errno 2] No such file or directory: 'h3'
/root
/root/build
CMake Warning:
  Ignoring extra path from command line:

   ".."


CMake Error: The source directory "/root" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.
make: *** No targets specified and no makefile found.  Stop.
make: *** No rule to make target 'install'.  Stop.


In [24]:
!pip install h3

In [25]:
!pip install osmnx

In [26]:
# Подключение к облачному хранилищу
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Начало работы

In [22]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import h3
import folium
import osmnx as ox
from shapely import wkt
from folium.plugins import HeatMap
from shapely.geometry import Polygon

In [30]:
path = '/content/drive/MyDrive/VTB/'
data = pd.read_excel(path + '2гис_коворк.xlsx')

In [34]:
coordinates = [[row['Широта'], row['Долгота']] for _, row in data[['Широта', 'Долгота']].iterrows()]

Визуализируем геск для первого адреса в выборке

In [35]:
def visualize_hexagons(hexagons, color="red", folium_map=None):

    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)

    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=20, tiles='cartodbpositron')
    else:
        m = folium_map

    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m

h3_address = h3.geo_to_h3(coordinates[0][0], coordinates[0][1],  9) # 9 - индекс, определяющий размер гексагона
visualize_hexagons([h3_address])


Перенесем данные карты Спб на карту

In [36]:
def visualize_polygons(geometry):

    lats, lons = get_lat_lon(geometry)

    m = folium.Map(location=[sum(lats)/len(lats), sum(lons)/len(lons)], zoom_start=13, tiles='cartodbpositron')

    overlay = gpd.GeoSeries(geometry).to_json()
    folium.GeoJson(overlay, name = 'boundary').add_to(m)

    return m

In [37]:
# выводим центроиды полигонов
def get_lat_lon(geometry):

    lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
    lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)
    return lat, lon

In [64]:
# выгрузим границы Питера из OSM
cities = ['Санкт-Петербург']
polygon_spb = ox.geocode_to_gdf('Санкт-Петербург, Россия')['geometry']
# посмотрим что получилось
visualize_polygons(polygon_spb)

<ipython-input-37-b46aea56d8de>:4: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
<ipython-input-37-b46aea56d8de>:5: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)


Сгенерим гексагоны внутри полигона:



In [66]:
def create_hexagons(geoJson):

    polyline = geoJson['coordinates'][0]

    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color="green")
    m.add_child(my_PolyLine)

    hexagons = list(h3.polyfill(geoJson, 8))
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=3,color='red')
        m.add_child(my_PolyLine)

    polylines_x = []
    for j in range(len(polylines)):
        a = np.column_stack((np.array(polylines[j])[:,1],np.array(polylines[j])[:,0])).tolist()
        polylines_x.append([(a[i][0], a[i][1]) for i in range(len(a))])

    polygons_hex = pd.Series(polylines_x).apply(lambda x: Polygon(x))

    return m, polygons_hex, polylines
# polygon_hex , polylines - геометрии гексагонов в разных форматах

# сгенерим гексагоны внутри полигона г. Краснодар
geoJson = json.loads(gpd.GeoSeries(polygon_spb).to_json())
geoJson = geoJson['features'][0]['geometry']
geoJson = {'type':'Polygon','coordinates': [np.column_stack((np.array(geoJson['coordinates'][0])[:, 1],
                                                      np.array(geoJson['coordinates'][0])[:, 0])).tolist()]}

m, polygons, polylines = create_hexagons(geoJson)
m


### Выгрузим данные с объектами Спб

#### Код

In [72]:
def osm_query(tag, city):
    gdf = ox.geometries_from_place(city, tag).reset_index()
    gdf['city'] = np.full(len(gdf), city.split(',')[0])
    gdf['object'] = np.full(len(gdf), list(tag.keys())[0])
    gdf['type'] = np.full(len(gdf), tag[list(tag.keys())[0]])
    gdf = gdf[['city', 'object', 'type', 'geometry']]
    print(gdf.shape)
    return gdf

 # Выгрузим интересующие нас категории объектов
tags = [
        {'building' : 'apartments'}, {'building' : 'detached'},
        {'building' : 'dormitory'}, {'building' : 'hotel'},
        {'building' : 'house'}, {'building' : 'semidetached_house'},
        {'building' : 'terrace'},  {'building' : 'commercial'},
        {'building' : 'office'},  {'building' : 'terrace'},
        {'building' : 'terrace'}, {'building':'retail'},
        {'building':'train_station'},
        {'highway' : 'bus_stop'}, {'footway':'crossing'},
        {'amenity':'cafe'}, {'amenity':'fast_food'},
        {'amenity':'restaurant'}, {'amenity':'college'},
        {'amenity':'language_school'},  {'amenity':'school'},
        {'amenity':'university'},  {'amenity':'atm'},
        {'amenity':'bank'},  {'amenity':'clinic'},
        {'amenity':'hospital'},  {'amenity':'pharmacy'},
        {'amenity':'theatre'},  {'amenity':'townhall'},
        {'amenity':'bench'},
       ]
cities = ['Санкт-Петербург, Россия']

gdfs = []
for city in cities:
    for tag in tags:
        gdfs.append(osm_query(tag, city))

# посмотрим что получилось
data_poi = pd.concat(gdfs)
data_poi.groupby(['city','object','type'], as_index = False).agg({'geometry':'count'})

# добавим координаты/центроиды
lat, lon = get_lat_lon(data_poi['geometry'])
data_poi['lat'] = lat
data_poi['lon'] = lon


<ipython-input-72-b9ba4efec0f1>:2: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  gdf = ox.geometries_from_place(city, tag).reset_index()


(14821, 4)


<ipython-input-72-b9ba4efec0f1>:2: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  gdf = ox.geometries_from_place(city, tag).reset_index()


(1600, 4)


<ipython-input-72-b9ba4efec0f1>:2: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  gdf = ox.geometries_from_place(city, tag).reset_index()


...

<ipython-input-72-b9ba4efec0f1>:2: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  gdf = ox.geometries_from_place(city, tag).reset_index()


(30, 4)


<ipython-input-72-b9ba4efec0f1>:2: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  gdf = ox.geometries_from_place(city, tag).reset_index()


(13892, 4)


<ipython-input-37-b46aea56d8de>:4: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
<ipython-input-37-b46aea56d8de>:5: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)


#### Результат

In [73]:
data_poi.groupby(['city','object','type'], as_index = False).agg({'geometry':'count'})

,city,object,type,geometry
0,Санкт-Петербург,amenity,atm,1780
1,Санкт-Петербург,amenity,bank,875
2,Санкт-Петербург,amenity,bench,13892
3,Санкт-Петербург,amenity,cafe,2919
4,Санкт-Петербург,amenity,clinic,468
5,Санкт-Петербург,amenity,college,166
6,Санкт-Петербург,amenity,fast_food,2350
7,Санкт-Петербург,amenity,hospital,115
8,Санкт-Петербург,amenity,language_school,97
9,Санкт-Петербург,amenity,pharmacy,1747


###Наложение объектов на гексы


#### Код

In [74]:
gdf_1 = gpd.GeoDataFrame(data_poi, geometry=gpd.points_from_xy(data_poi.lon, data_poi.lat))

gdf_2 = pd.DataFrame(polygons, columns = ['geometry'])
gdf_2['polylines'] = polylines
gdf_2['geometry'] = gdf_2['geometry'].astype(str)
geometry_uniq = pd.DataFrame(gdf_2['geometry'].drop_duplicates())
geometry_uniq['id'] = np.arange(len(geometry_uniq)).astype(str)
gdf_2 = gdf_2.merge(geometry_uniq, on = 'geometry')
gdf_2['geometry'] = gdf_2['geometry'].apply(wkt.loads)
gdf_2 = gpd.GeoDataFrame(gdf_2, geometry='geometry')

itog_table = gpd.sjoin(gdf_2, gdf_1, how='left', op='intersects')
itog_table = itog_table.dropna()
itog_table.head()

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-74-e92babab3ce1>:12: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  itog_table = gpd.sjoin(gdf_2, gdf_1, how='left', op='intersects')


,geometry,polylines,id,index_right,city,object,type,lat,lon
0,"POLYGON ((30.29331 59.92846, 30.29865 59.92507...","[(59.9284642442585, 30.293309754883232), (59.9...",0,2633.0,Санкт-Петербург,footway,crossing,59.926681,30.305413
0,"POLYGON ((30.29331 59.92846, 30.29865 59.92507...","[(59.9284642442585, 30.293309754883232), (59.9...",0,813.0,Санкт-Петербург,amenity,cafe,59.926856,30.306900
0,"POLYGON ((30.29331 59.92846, 30.29865 59.92507...","[(59.9284642442585, 30.293309754883232), (59.9...",0,4282.0,Санкт-Петербург,footway,crossing,59.926902,30.305931
0,"POLYGON ((30.29331 59.92846, 30.29865 59.92507...","[(59.9284642442585, 30.293309754883232), (59.9...",0,160.0,Санкт-Петербург,amenity,cafe,59.926932,30.306302
0,"POLYGON ((30.29331 59.92846, 30.29865 59.92507...","[(59.9284642442585, 30.293309754883232), (59.9...",0,2627.0,Санкт-Петербург,footway,crossing,59.926995,30.305405


#### Результат

In [75]:
itog_table.head()

,geometry,polylines,id,index_right,city,object,type,lat,lon
0,"POLYGON ((30.29331 59.92846, 30.29865 59.92507...","[(59.9284642442585, 30.293309754883232), (59.9...",0,2633.0,Санкт-Петербург,footway,crossing,59.926681,30.305413
0,"POLYGON ((30.29331 59.92846, 30.29865 59.92507...","[(59.9284642442585, 30.293309754883232), (59.9...",0,813.0,Санкт-Петербург,amenity,cafe,59.926856,30.306900
0,"POLYGON ((30.29331 59.92846, 30.29865 59.92507...","[(59.9284642442585, 30.293309754883232), (59.9...",0,4282.0,Санкт-Петербург,footway,crossing,59.926902,30.305931
0,"POLYGON ((30.29331 59.92846, 30.29865 59.92507...","[(59.9284642442585, 30.293309754883232), (59.9...",0,160.0,Санкт-Петербург,amenity,cafe,59.926932,30.306302
0,"POLYGON ((30.29331 59.92846, 30.29865 59.92507...","[(59.9284642442585, 30.293309754883232), (59.9...",0,2627.0,Санкт-Петербург,footway,crossing,59.926995,30.305405


### Распределение объектов на карте

#### Код

In [76]:
def create_choropleth(data, json, columns, legend_name, feature, bins):

    lat, lon = get_lat_lon(data['geometry'])

    m = folium.Map(location=[sum(lat)/len(lat), sum(lon)/len(lon)], zoom_start=13, tiles='cartodbpositron')

    folium.Choropleth(
        geo_data=json,
        name="choropleth",
        data=data,
        columns=columns,
        key_on="feature.id",
        fill_color="YlGn",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=legend_name,
        nan_fill_color = 'black',
        bins = bins

    ).add_to(m)

    folium.LayerControl().add_to(m)

    return m

# подготовим данные
itog_table['geometry'] = itog_table['geometry'].astype(str) #для groupby
itog_table['id'] = itog_table['id'].astype(str) #для Choropleth
agg_all = itog_table.groupby(['geometry','type','id'], as_index = False).agg({'lat':'count'}).rename(columns = {'lat':'counts'})
agg_all['geometry'] = agg_all['geometry'].apply(wkt.loads) #возвращаем формат геометрий

agg_all_cafe = agg_all.query("type == 'cafe'")[["geometry","counts",'id']]
agg_all_cafe['id'] = agg_all_cafe['id'].astype(str)
data_geo_1 = gpd.GeoSeries(agg_all_cafe.set_index('id')["geometry"]).to_json()


/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1537: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


#### Результат

In [89]:
cut_off = 30
agg_all_cafe_cut = agg_all_cafe[agg_all_cafe['counts'] < cut_off]

create_choropleth(agg_all_cafe_cut, data_geo_1, ["id","counts"], 'Cafe counts', 'counts', 10)

<ipython-input-37-b46aea56d8de>:4: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
<ipython-input-37-b46aea56d8de>:5: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)
